In [10]:
import pandas as pd
import numpy as np
import json

In [11]:
df = pd.read_csv('test.csv')
with open('categories.json', 'r', encoding='utf-8') as file:
    categories = json.load(file)

In [12]:
df.drop(['ID', 'Customer_ID', 'Name', 'SSN'], axis=1, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     50000 non-null  object 
 1   Age                       50000 non-null  object 
 2   Occupation                50000 non-null  object 
 3   Annual_Income             50000 non-null  object 
 4   Monthly_Inhand_Salary     42502 non-null  float64
 5   Num_Bank_Accounts         50000 non-null  int64  
 6   Num_Credit_Card           50000 non-null  int64  
 7   Interest_Rate             50000 non-null  int64  
 8   Num_of_Loan               50000 non-null  object 
 9   Type_of_Loan              44296 non-null  object 
 10  Delay_from_due_date       50000 non-null  int64  
 11  Num_of_Delayed_Payment    46502 non-null  object 
 12  Changed_Credit_Limit      50000 non-null  object 
 13  Num_Credit_Inquiries      48965 non-null  float64
 14  Credit

In [14]:
df.select_dtypes('O').info()
df.select_dtypes('O').head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Month                    50000 non-null  object
 1   Age                      50000 non-null  object
 2   Occupation               50000 non-null  object
 3   Annual_Income            50000 non-null  object
 4   Num_of_Loan              50000 non-null  object
 5   Type_of_Loan             44296 non-null  object
 6   Num_of_Delayed_Payment   46502 non-null  object
 7   Changed_Credit_Limit     50000 non-null  object
 8   Credit_Mix               50000 non-null  object
 9   Outstanding_Debt         50000 non-null  object
 10  Credit_History_Age       45530 non-null  object
 11  Payment_of_Min_Amount    50000 non-null  object
 12  Amount_invested_monthly  47729 non-null  object
 13  Payment_Behaviour        50000 non-null  object
 14  Monthly_Balance          49438 non-nul

,Month,Age,Occupation,Annual_Income,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Changed_Credit_Limit,Credit_Mix,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,September,23,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,11.27,Good,809.98,22 Years and 9 Months,No,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,October,24,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",9,13.27,Good,809.98,22 Years and 10 Months,No,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,November,24,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",4,12.27,Good,809.98,NaN,No,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,December,24_,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,11.27,Good,809.98,23 Years and 0 Months,No,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,September,28,_______,34847.84,1,Credit-Builder Loan,1,5.42,Good,605.03,27 Years and 3 Months,No,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [15]:
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')).replace([''], np.NaN)
df['Month'] = pd.to_datetime(df.Month, format='%B').dt.month
df['Age'] = df.Age.astype(int)
df['Annual_Income'] = df.Annual_Income.astype(float)
df['Num_of_Loan'] = df.Num_of_Loan.astype(int)
df['Num_of_Delayed_Payment'] = df.Num_of_Delayed_Payment.astype(float)
df['Changed_Credit_Limit'] = df.Changed_Credit_Limit.astype(float)
df['Outstanding_Debt'] = df.Outstanding_Debt.astype(float)
df['Amount_invested_monthly'] = df.Amount_invested_monthly.astype(float)
df['Monthly_Balance'] = df.Monthly_Balance.astype(float)

In [16]:
def month_text_to_number(x):
    if pd.notnull(x):
        num1 = int(x.split(' ')[0])
        num2 = int(x.split(' ')[3])
        return (num1*12)+num2
    else:
        return x
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: month_text_to_number(x)).astype(float)

In [17]:
df['Type_of_Loan'] = df['Type_of_Loan'].str.replace(', and', ',')
loans_set = categories['loans']
df['Type_of_Loan'] = df['Type_of_Loan'].map(lambda line: np.array([1 if not pd.isnull(line) and x in line.split(', ') else 0 for x in loans_set]))
for item in loans_set:
    df[item] = df['Type_of_Loan'].map(lambda x: x[loans_set.index(item)])
df.drop(['Type_of_Loan'], axis=1, inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     50000 non-null  int64  
 1   Age                       50000 non-null  int32  
 2   Occupation                46562 non-null  object 
 3   Annual_Income             50000 non-null  float64
 4   Monthly_Inhand_Salary     42502 non-null  float64
 5   Num_Bank_Accounts         50000 non-null  int64  
 6   Num_Credit_Card           50000 non-null  int64  
 7   Interest_Rate             50000 non-null  int64  
 8   Num_of_Loan               50000 non-null  int32  
 9   Delay_from_due_date       50000 non-null  int64  
 10  Num_of_Delayed_Payment    46502 non-null  float64
 11  Changed_Credit_Limit      48941 non-null  float64
 12  Num_Credit_Inquiries      48965 non-null  float64
 13  Credit_Mix                40195 non-null  object 
 14  Outsta

In [19]:
df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Occupation             46562 non-null  object
 1   Credit_Mix             40195 non-null  object
 2   Payment_of_Min_Amount  50000 non-null  object
 3   Payment_Behaviour      50000 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [20]:
df['Occupation'].value_counts(dropna=False)

NaN              3438
Lawyer           3324
Engineer         3212
Architect        3195
Mechanic         3168
Developer        3146
Accountant       3133
Media_Manager    3130
Scientist        3104
Teacher          3103
Entrepreneur     3103
Journalist       3037
Doctor           3027
Manager          3000
Musician         2947
Writer           2933
Name: Occupation, dtype: int64

In [21]:
df['Credit_Mix'].value_counts(dropna=False)

Standard    18379
Good        12260
NaN          9805
Bad          9556
Name: Credit_Mix, dtype: int64

In [22]:
df['Payment_of_Min_Amount'].value_counts(dropna=False)

Yes    26158
No     17849
NM      5993
Name: Payment_of_Min_Amount, dtype: int64

In [23]:
df['Payment_Behaviour'].value_counts(dropna=False)

Low_spent_Small_value_payments      12694
High_spent_Medium_value_payments     8922
High_spent_Large_value_payments      6844
Low_spent_Medium_value_payments      6837
High_spent_Small_value_payments      5651
Low_spent_Large_value_payments       5252
!@9#%8                               3800
Name: Payment_Behaviour, dtype: int64

In [24]:
df['Occupation'] = df.Occupation.apply(lambda x: 'Unknown' if pd.isnull(x) else x)
df['Credit_Mix'] = df.Credit_Mix.apply(lambda x: 'Unknown' if pd.isnull(x) else x)
df['Payment_of_Min_Amount'] = df.Payment_of_Min_Amount.apply(lambda x: 'Unknown' if x == 'NM' else x)
df['Payment_Behaviour'] = df.Payment_Behaviour.apply(lambda x: 'Unknown' if x == '!@9#%8' else x)

In [27]:
df.dropna(inplace=True)

In [28]:
df.select_dtypes('O').columns

Index(['Occupation', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Payment_Behaviour'],
      dtype='object')

In [29]:
occupations = categories['occupations']
df['Occupation'] = df.Occupation.apply(lambda x: occupations.index(x) if x in occupations else occupations.index('Unknown'))

payment_min_amounts = categories['payment_min_amounts']
df['Payment_of_Min_Amount'] = df.Payment_of_Min_Amount.apply(lambda x: payment_min_amounts.index(x) if x in payment_min_amounts else payment_min_amounts.index('Unknown'))

payment_behaviours = categories['payment_behaviours']
df['Payment_Behaviour'] = df.Payment_Behaviour.apply(lambda x: payment_behaviours.index(x) if x in payment_behaviours else payment_behaviours.index('Unknown'))

credit_mixs = categories['credit_mixs']
df['Credit_Mix'] = df.Credit_Mix.apply(lambda x: credit_mixs.index(x) if x in credit_mixs else credit_mixs.index('Unknown'))

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32565 entries, 0 to 49999
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     32565 non-null  int64  
 1   Age                       32565 non-null  int32  
 2   Occupation                32565 non-null  int64  
 3   Annual_Income             32565 non-null  float64
 4   Monthly_Inhand_Salary     32565 non-null  float64
 5   Num_Bank_Accounts         32565 non-null  int64  
 6   Num_Credit_Card           32565 non-null  int64  
 7   Interest_Rate             32565 non-null  int64  
 8   Num_of_Loan               32565 non-null  int32  
 9   Delay_from_due_date       32565 non-null  int64  
 10  Num_of_Delayed_Payment    32565 non-null  float64
 11  Changed_Credit_Limit      32565 non-null  float64
 12  Num_Credit_Inquiries      32565 non-null  float64
 13  Credit_Mix                32565 non-null  int64  
 14  Outsta

In [31]:
df.to_csv('clean_test.csv', index=False)